<a href="https://colab.research.google.com/github/himanshu2s/EmployeeRegistration/blob/master/Solve_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [1]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000002MLxxMAG'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])


In [2]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [5]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/13m9mUF9TdVg-ngm9x44GOeZwPtxxNeW9agEB6NAV2Vw




```
# This is formatted as code
```

# Task 1. Classify emails

In [20]:
# Use this section to classify emails as per the email subject/content and put the results in a google drive sheet.

from google.colab import drive

def classify_email(subject, message):
    prompt = f"""
    Given the following email, classify it as either a "product inquiry" or an "order request":

    Subject: {subject}
    Message: {message}

    Respond with only one of the two categories: "product inquiry" or "order request".
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10
    )
    return response.choices[0].message.content.strip()

# Classify emails
emails_df["category"] = emails_df.apply(lambda row: classify_email(row["subject"], row["message"]), axis=1)

# Mount Google Drive
drive.mount('/content/drive')

# Define file path
output_file_name = "classified_emails.xlsx"
output_path = f'/content/drive/MyDrive/{output_file_name}'

# Save results
emails_df.to_excel(output_path, index=False)

print(f"Classification completed. Check '{output_path}'.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Classification completed. Check '/content/drive/MyDrive/classified_emails.xlsx'.


In [21]:
# Use this section to get a sharable Google Sheet link.

import os
import pandas as pd
import time
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Authenticate Google Drive API
auth.authenticate_user()
drive_service = build('drive', 'v3')

try:
    # Search for the file in Google Drive
    response = drive_service.files().list(q=f"name='{output_file_name}'", fields="files(id)").execute()

    if not response.get('files'):
        print("⚠️ File not found in Drive. Uploading manually...")

        # Upload file to Google Drive
        file_metadata = {
            'name': f"{output_file_name}",
            'parents': ['root']  # Saves to My Drive
        }
        media = MediaFileUpload(output_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        upload_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        file_id = upload_file.get('id')

    else:
        file_id = response['files'][0]['id']

    # Make the file publicly accessible
    drive_service.permissions().create(
        fileId=file_id,
        body={'role': 'reader', 'type': 'anyone'},
    ).execute()

    # Print the sharable link
    sharable_link = f"https://docs.google.com/spreadsheets/d/{file_id}/"
    print(f"🔗 Sharable link: {sharable_link}")

except Exception as e:
    print(f"🚨 Error: {e}")

🔗 Sharable link: https://docs.google.com/spreadsheets/d/1-8DXRBrQbiYt1i_HcPCg_Rz95RdHN9t_/


# Task 2. Process order requests

In [27]:
import pandas as pd
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from openai import OpenAI

# Authenticate with Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Replace with your Google Sheet ID
print(file_id)
spreadsheet = gc.open_by_key(file_id)
# spreadsheet = gc.open_by_key("1-8DXRBrQbiYt1i_HcPCg_Rz95RdHN9t_")
# spreadsheet = gc.open_by_url(sharable_link)

# Load data from Google Sheets
products_ws = spreadsheet.worksheet("products")
emails_ws = spreadsheet.worksheet("emails")
classified_ws = spreadsheet.worksheet("email-classification")

# Convert sheets to DataFrames
products_df = pd.DataFrame(products_ws.get_all_records())
emails_df = pd.DataFrame(emails_ws.get_all_records())
classified_df = pd.DataFrame(classified_ws.get_all_records())

# Merge emails with classifications
emails_df = emails_df.merge(classified_df, on="email_id")

# Initialize OpenAI client
client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj000002MLxxMAG'
)

# Extract order-related emails
order_requests = emails_df[emails_df["category"] == "order request"]

# Order processing results
order_status_list = []
order_responses = []

for _, row in order_requests.iterrows():
    email_id = row["email_id"]
    message = row["message"]

    # Extract requested product details using GPT-4o
    prompt = f"""
    Extract product ID and requested quantity from the following email:
    "{message}"
    Respond in JSON format with keys: "product_id" and "quantity".
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50
    )

    try:
        order_data = eval(response.choices[0].message.content)  # Convert JSON string to dictionary
        product_id = order_data["product_id"]
        quantity = int(order_data["quantity"])
    except:
        continue  # Skip invalid responses

    # Check stock availability
    product_info = products_df[products_df["product_id"] == product_id]

    if product_info.empty:
        status = "out of stock"
        response_text = f"Sorry, we could not find the requested product ({product_id}). Please check the product details and try again."
    else:
        stock_available = int(product_info["stock"].values[0])

        if quantity <= stock_available:
            status = "created"
            new_stock = stock_available - quantity
            products_df.loc[products_df["product_id"] == product_id, "stock"] = new_stock
            response_text = f"Your order for {quantity} units of {product_info['name'].values[0]} has been successfully placed."
        else:
            status = "out of stock"
            response_text = f"Unfortunately, we only have {stock_available} units of {product_info['name'].values[0]} available. Please adjust your order or wait for restocking."

    # Append order processing results
    order_status_list.append([email_id, product_id, quantity, status])
    order_responses.append([email_id, response_text])

# Convert results to DataFrame
order_status_df = pd.DataFrame(order_status_list, columns=["email_id", "product_id", "quantity", "status"])
order_response_df = pd.DataFrame(order_responses, columns=["email_id", "response"])

# Save order status back to Google Sheets
order_status_ws = spreadsheet.worksheet("order-status")
order_status_ws.update([order_status_df.columns.values.tolist()] + order_status_df.values.tolist())

# Save order responses to Google Sheets
order_response_ws = spreadsheet.worksheet("order-response")
order_response_ws.update([order_response_df.columns.values.tolist()] + order_response_df.values.tolist())

print("✅ Order processing completed and updated in Google Sheets.")


1-8DXRBrQbiYt1i_HcPCg_Rz95RdHN9t_


APIError: APIError: [400]: This operation is not supported for this document

# Task 3. Handle product inquiry